# 최종 프로젝트
- 3일차

- 리소스 사용 계획서 작성
- WBS 작성
  >> 피해서버 구성
  >> 공격방식 선정
  >> 일부 일정 수정
- 요구기능정의서 작성
- 환경 분석 보고서 작성
- 환경 구성
  >> ubuntu 세팅
  >> kali 세팅
- splunk 라이선스 신청
- 구성도 제작
  >> https://www.canva.com/design/DAG2Z9UO7ng/Z-WPvmGPCXj3BUaH8ce7lQ/edit

## 인터페이스 요구사항

### 사용자 인터페이스 (UI)

- 통합 관제 대시보드 (핵심): 관제 인력이 24시간 모니터링할 메인 화면으로, 실시간 경보, 핵심 보안 지표(EPS, 공격 Top5), 시스템 헬스체크 등 TO-BE 통합 모니터링 요건을 충족해야 합니다.
- AI 분석 결과 뷰: 관제사가 경보(Alert)를 클릭했을 때, Gemini API가 분석한 '위협 요약', '공격 단계', '대응 권고안'을 즉시 확인할 수 있도록 직관적으로 시각화해야 합니다.
- ML 피드백 및 보고서 뷰: MLTK가 탐지한 이상징후에 대해 분석가가 '정/오탐' 피드백을 입력할 수 있어야 하며, ISMS-P감사를 위한 '자동 리포트' 생성 및 조회 기능을 제공해야 합니다.
- 역할 기반 접근(RBAC): 사용자의 역할(관제사, 분석가, 관리자)에 따라 접근 가능한 메뉴와 데이터가 차등 적용(RBAC)되어야 합니다.

### 시스템 인터페이스 (Data & API)

-로그 수집 인터페이스 (Ingress): 현황[cite: 2.1]에 명시된 모든 인프라(pfSense, ModSecurity, Snort, 서버, Cisco, NAC, DLP, EPP 등)로부터 Syslog(v1 포함), WMI, Agent 등 다양한 방식으로 로그를 수집할 수 있어야 합니다.
- AI 지식 수집 인터페이스 (Ingress): MLTK/Gemini 분석을 위해 외부 위협 인텔리전스(TI), CVE 정보, 내부 보안 정책 데이터를 API 또는 파일로 수집할 수 있어야 합니다.
- Gemini API 연동 (Egress - 핵심): Splunk가 정규화한 로그와 프롬프트를 Gemini API 로 안전하게 전송하고, 분석된 JSON 응답을 수신할 수 있어야 합니다.
- 알림 및 확장 인터페이스 (Egress): 고위험 경보 발생 시 담당자에게 SMTP(이메일) 또는 Webhook(메신저)으로 알림을 전송해야 하며, 향후 자동 대응을 위한 API 연동(pfSense, NAC)이 가능해야 합니다.

## 데이터 요구사항

본 섹션은 “지능형 SIEM 구축” 프로젝트의 핵심 자산인 데이터의 요건을 정의합니다. 모든 데이터 수집, 처리, 보관 및 보안은 **Splunk MLTK**와 **Gemini API**를 활용한 지능형 분석 및 탐지 고도화를 전제로 합니다.

### 데이터 수집 및 연동

시스템은 상관분석 및 AI 학습에 필요한 모든 내부/외부 데이터를 실시간으로 수집 및 연동해야 합니다.

- **인프라 로그:** 현황 분석[cite: 2.1]에 명시된 370여 대의 모든 자산 로그를 수집합니다.
- **AI 학습용 지식 베이스:** ML/LLM 분석 컨텍스트 확보를 위해 다음 데이터를 수집합니다.
    - 외부 위협 인텔리전스(TI) 피드 (IP/도메인/해시 평판)
    - 공개 취약점 정보 (CVE,OWASP 등)
    - 내부 보안 정책 및 ISMS-P[cite: 8.3] 감사 항목
- **분석가 피드백 데이터:** 분석가의 **정·오탐 판별 결과**[cite: FR-정탐/오탐 판별]를 MLTK 모델 재학습(Feedback Loop)을 위한 데이터 소스로 다시 수집(Feed)해야 합니다.

### 데이터 처리 및 성능

시스템은 대용량 로그를 지연 없이 처리하고 다양한 환경을 지원해야 합니다.

- **처리 용량:** 요구사항[cite: FR-100]에 명시된 **10,000 EPS** 이상의 대용량 로그를 실시간으로 수집, 처리, 인덱싱할 수 있어야 합니다.
- **수집 호환성:** 제약사항[cite: 8.1]의 **Syslog v1**을 포함하여 WMI[cite: FR-100], RESTful API[cite: FR-100] 등 이기종 장비의 다양한 프로토콜을 모두 지원해야 합니다.
- **처리 지연:** 로그 발생 시점부터 SIEM 검색 및 분석 가능 시점까지의 지연은 **3분 이내**를 보장합니다.[cite: 2.3 TO-BE, 1.4]
- **성능 분리:** **MLTK 학습** 및 **Gemini API** 호출 등 리소스 소모가 큰 AI 연산 작업이 실시간 로그 수집 및 인덱싱 성능에 영향을 주지 않도록 자원을 분리하여 설계해야 합니다.[cite: NFR-확장성]

### 데이터 정규화 및 분석

수집된 원본 로그(Raw Log)는 AI가 즉시 분석 가능한 표준화된 데이터로 가공되어야 합니다.

- **데이터 표준화(CIM):** 모든 이기종 로그는 **Splunk CIM(공통 정보 모델)**을 기준으로 정규화(Parsing)되어야 합니다. 이는 모든 상관분석[cite: 2.2 상관분석] 및 AI 분석의 전제 조건입니다.
- **AI 입력 데이터 :**
    - *(MLTK):* CIM으로 표준화된 로그는 MLTK의 이상징후 탐지 모델의 입력값으로 즉시 활용 가능해야 합니다.
    - *(Gemini):* 탐지된 이벤트 로그, 연관된 TI 정보, 사내 정책을 조합하여 **Gemini API 요청**에 맞는 최적화된 프롬프트를 동적으로 생성해야 합니다.
- **AI 출력 데이터 :**
    - *(Gemini):* **Gemini API가 반환하는 JSON 또는 텍스트 응답**을 파싱(Parsing)하여, '위협 요약', '권고 조치', '예상 공격 단계' 등의 필드로 분리하고 Splunk 티켓(Case)에 저장해야 합니다.

### 데이터 저장 및 보관

로그 데이터와 AI 자산은 법적/운영적 요구사항에 따라 안전하게 보관되어야 합니다.

- **로그 보관 주기:** **ISMS-P 규정**[cite: 8.3] 및 비기능 요구사항[cite: NFR-절차 및 이력 보존성]에 따라 모든 원본 로그와 대응 이력을 **최소 1년 이상** 위변조 불가능한 상태로 보관합니다.
- **저장 방식:** 예산 제약[cite: 8.2] 내에서 실시간 검색 성능(Hot/Warm)과 장기 보관 비용(Cold)을 모두 만족하는 **계층형 저장** 방식을 적용하여 성능과 비용을 최적화합니다.
- **AI 자산 관리:** LLM 지식베이스 등 핵심 AI 자산은 로그와 별도로 안전하게 백업 및 버전 관리합니다.

### 데이터 보안 및 접근 제어

데이터는 법적 요구사항을 준수하며 엄격한 통제하에 관리되어야 합니다.

- **개인정보보호:** **개인정보보호법**[cite: 8.3]에 따라 로그 수집(파싱) 시 모든 식별 가능한 민감 정보는 **마스킹** 처리해야 합니다.
- **권한 관리(RBAC):** 관제사, 분석가, AI 개발자 등 **역할 기반 접근 제어(RBAC)**[cite: FR-사용자 권한 관리]를 통해 데이터 조회, 설정, 모델 학습 권한을 엄격히 분리합니다.
- **API 키 관리:** **Gemini API 키**는 암호화되어 안전하게 보관되어야 하며, 시스템 외부로 노출되지 않도록 엄격히 통제해야 합니다.
- **감사 추적:** 모든 사용자의 시스템 내 행위(검색, 설정 변경, API 호출 등)는 감사 로그로 기록하여 1년 이상 보관합니다.[cite: NFR-감사 및 추적성]

## 제약사항

본 프로젝트는 다음의 기술적, 관리적, 법적 한계 내에서 수행되어야 한다.

### 기술적 제약사항

- **기존 가상화 환경 활용:** 신규 SIEM(Splunk) 시스템은 물리적 어플라이언스가 아닌, 현재 운영 중인 **VMware ESXi 6.7 환경**에 가상 머신(VM) 형태로 구축해야 한다.
- **기존 보안 솔루션 연동:** **FortiGate, Snort, WEBFRONT-K, AhnLab EPP, Genian NAC, Somansa DLP** 등 기존에 도입된 보안 솔루션의 교체를 전제하지 않으며, SIEM은 이 모든 이기종 장비의 로그를 수집할 수 있어야 한다.
- **네트워크 성능 보장:** SIEM 로그 수집 트래픽(Syslog, Agent 등)은 기존 업무망 및 서비스망의 **네트워크 운영에 영향을 주지 않아야** 하며, 트래픽 폭증 시 QoS(품질 관리) 정책을 적용한다.
- **vCenter 미사용 환경:** 현재 중앙 관리용 **vCenter가 부재**한 환경을 고려하여, SIEM 배포 및 리소스 모니터링 방안을 수립해야 한다.

### 관리적 제약사항

- **운영 인력 제한:** SIEM 도입 후에도 관제는 **기존 3인의 인력**으로 운영하는 것을 원칙으로 한다. 따라서 SIEM은 자동화된 경보 및 직관적인 대시보드를 제공하여 최소 인력으로 운영 가능해야 한다.
- **프로젝트 기간:** 본 프로젝트는 계약일로부터 **6개월** 이내에 분석 시나리오 적용 및 운영자 교육을 포함한 모든 구축을 완료해야 한다.
- **프로젝트 예산:** H/W 도입, S/W 라이선스, 구축 공수, 교육을 포함한 총예산은 **3억 원** 이내로 한정한다.

### 법적/규제 제약사항

- **ISMS-P 규정 준수:** 로그 수집 및 분석 체계는 **ISMS-P 인증 요구사항을 충족**해야 한다.
- **로그 보관 주기:** 관련 법규(개인정보보호법 등) 및 ISMS-P 인증 기준을 충족하기 위해, 수집된 모든 원본 로그는 **최소 6개월 이상** 위변조가 불가능한 상태로 보관되어야 한다.
- **개인정보보호:** 로그에 포함될 수 있는 개인정보(예: DB 접근 로그)는 식별 불가능하도록 마스킹 또는 암호화되어야 하며, 인가된 담당자 외에는 접근을 엄격히 통제해야 한다.